#Implementation of Ankh library from Google.

In [ ]:
#@title Install and import dependencies. { display-mode: "form" }
%%capture
!pip3 install torch torchvision torchaudio transformers sentencepiece accelerate --extra-index-url https://download.pytorch.org/whl/cu116
!python -m pip install ankh

from transformers import BertForMaskedLM, BertTokenizer, pipeline
from transformers import AutoModel, AutoTokenizer
from sklearn.manifold import TSNE
from google.colab import drive
import torch, os, pandas as pd, random, matplotlib.pyplot as plt, numpy as np
import seaborn as sns
import time, random, re, shutil
import ankh

drive.mount('/content/drive/')

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using device: {}".format(device))

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/msc-project-mbalmf01/all_paired/230716_human_paired_seqs.csv', dtype={'Run':np.str_}, index_col=0)
prott5 = pd.read_csv('230716_scfv_10000.csv', index_col=0)

names = prott5['seq']
df = df[df['seq_id'].isin(names)]

protein_sequences = [df['scfv'].iloc[1], df['scfv'].iloc[2]]

Index(['Unnamed: 1', 'sequence_id_heavy', 'ANARCI_status_heavy',
       'sequence_heavy', 'sequence_alignment_aa_heavy', 'v_call_heavy',
       'd_call_heavy', 'j_call_heavy', 'sequence_id_light',
       'ANARCI_status_light', 'sequence_light', 'v_call_light', 'j_call_light',
       'sequence_alignment_aa_light', 'Run', 'seq_id', 'scfv'],
      dtype='object')

In [ ]:
path = '/content/drive/MyDrive/msc-project-mbalmf01/msc-project-source-code-files-22-23-mbalmf01/notebooks'
os.chdir(path)

from plm_manipulation import start_ankh, process_seqs, batch_embed, redux_fit

model, tokenizer = start_ankh


In [ ]:
tensor_df = batch_embed(df=df, prot_col='scfv', seq_id='seq_id', batch_size=100, model=model, tokenizer=tokenizer)

In [ ]:
tensor_df.shape

(9955, 1537)

In [ ]:
tensor_df.to_csv('230805_ankh_tensor_10000.csv', index=False)